# Exercise 2.4 - Hyperparameter Optimisation (CNN)

This script is organised using the following structure:
1. **Importing Libraries and Data**
2. **Reshape Data for Bayesian Optimisation**
3. **Split the Data**
4. **LSTM Neural Network and Bayesian Optimiser Function**
5. **Running CNN with Hyperparameters from Bayesian Optimisation**
6. **Confusion Matrix**

## 1. Importing Libraries and Data

In [4]:
import pandas as pd
import numpy as np
import os
import operator
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Packages and functions specific to models and optimisation
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(negative_slope=0.1) # alpha deprecated - use negative_slope instead
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [6]:
path = r'C:\Users\kyles\CareerFoundary\Machine Learning\Achievement 2'

In [8]:
# Import unscaled weather data
df_weather = pd.read_csv(os.path.join(path, '02 Data', 'Cleaned_Weather_Data.csv'))
df_weather.drop(columns=['Unnamed: 0'], inplace = True)
# Check
df_weather

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.00,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.00,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.2

In [10]:
# Import 'pleasant weather' data
df_answers = pd.read_csv(os.path.join(path, '02 Data', 'Pleasant_Weather_Prediction_Answers.csv'))
df_answers.drop(columns=['DATE'], inplace = True) # Assuming we don't need dates for this
# Check
df_answers

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2. Reshape Data for Bayesian Optimisation

Bayesian Optimisation requires a slightly different form for both data sets before they are split:
- X shape will need to be (22950, 15, 9)
- y shape will need to be (22950,)

In [14]:
# Start with renaming and turning data into arrays
X = np.array(df_weather)
y = np.array(df_answers)

# Check current shapes
print(f'X shape is currently: {X.shape}')
print(f'y shape is currently: {y.shape}')

X shape is currently: (22950, 135)
y shape is currently: (22950, 15)


In [16]:
# Change shape of X
X = X.reshape(-1, 15, 9)
X.shape # verify

(22950, 15, 9)

In [18]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multilabel-indicator'

In [20]:
# Change shape of y
y = np.argmax(y, axis = 1)
y # look at data to see if it seems correct

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
# Verify X and y shapes
print(f'X shape is currently: {X.shape}')
print(f'y shape is currently: {y.shape}')

X shape is currently: (22950, 15, 9)
y shape is currently: (22950,)


Fantastic

In [25]:
# Check unique values in y
unique_classes = np.unique(y)
print(unique_classes)
print(len(unique_classes)) 

[ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
14


Perhaps there is an error with Sonnblick (having only one type of value)

## 3. Split the Data

In [29]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) # Can also control size with train_size before random_state

In [31]:
# Check sizes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


## 4. LSTM Neural Network and Bayesian Optimiser Function

In [34]:
# Setup essential/non-changing parameters
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [52]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD = {
                    'Adam': Adam(learning_rate=learning_rate),
                    'SGD': SGD(learning_rate=learning_rate),
                    'RMSprop': RMSprop(learning_rate=learning_rate),
                    'Adadelta': Adadelta(learning_rate=learning_rate),
                    'Adagrad': Adagrad(learning_rate=learning_rate),
                    'Adamax': Adamax(learning_rate=learning_rate),
                    'Nadam': Nadam(learning_rate=learning_rate),
                    'Ftrl': Ftrl(learning_rate=learning_rate)
                 }
    activationL = ['sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def lstm_model():
        model = Sequential()
        model.add(LSTM(neurons, activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=lstm_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [58]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation': (0, 6),  # Adjusted activation range
    'optimizer': (0, 7),    # Adjusted optimizer range
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/25
15/15 - 9s - 626ms/step - accuracy: 0.6105 - loss: 2.7013
Epoch 2/25
15/15 - 1s - 88ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 86ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 87ms/step - accuracy: 0.6440 - loss: 2.6943
Epoch 5/25
15/15 - 1s - 99ms/step - accuracy: 0.6440 - loss: 2.6918
Epoch 6/25
15/15 - 1s - 96ms/step - accuracy: 0.6440 - loss: 2.6895
Epoch 7/25
15/15 - 3s - 190ms/step - accuracy: 0.6440 - loss: 2.6875
Epoch 8/25
15/15 - 2s - 155ms/step - accuracy: 0.6440 - loss: 2.6855
Epoch 9/25
15/15 - 1s - 94ms/step - accuracy: 0.6440 - loss: 2.6837
Epoch 10/25
15/15 - 3s - 189ms/step - accurac

ValueError: Input y contains NaN.

In [59]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'selu',
 'batch_size': 370,
 'dropout': 0.18182496720710062,
 'dropout_rate': 0.055021352956030146,
 'epochs': 29,
 'kernel': 2.049512863264476,
 'layers1': 2,
 'layers2': 2,
 'learning_rate': 0.6157343657751557,
 'neurons': 23,
 'normalization': 0.29214464853521815,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x24c1e628850>}

## 5. Running LSTM with Hyperparameters from Bayesian Optimisation

In [62]:
# Put the y_test set back into a one-hot configuration
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [64]:
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

y_train_one_hot shape: (17212, 15)


In [66]:
# Construct model with optimised hyperparameters
epochs = 29
batch_size = 370

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train_one_hot[0])

layers1 = 2
layers2 = 2
activation = 'selu'
kernel = int(round(2.049512863264476))
neurons = 23
normalization = 0.29214464853521815
dropout = 0.18182496720710062
dropout_rate = 0.055021352956030146
optimizer = Adadelta(learning_rate=0.6157343657751557)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(LSTM(neurons, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [68]:
model.summary()

Model: "sequential_90"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_90 (LSTM)                  │ (None, 23)             │         3,036 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_425 (Dense)               │ (None, 23)             │           552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_426 (Dense)               │ (None, 23)             │           552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_427 (Dense)               │ (None, 23)             │           552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_428 (Dense)               │ (None, 23)             │           552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_429 (Dense)               │ (None, 15)             │           360 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,604 (21.89 KB)

 Trainable params: 5,604 (21.89 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [72]:
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/29
47/47 - 11s - 237ms/step - accuracy: 0.5372 - loss: 1.6674
Epoch 2/29
47/47 - 1s - 17ms/step - accuracy: 0.6441 - loss: 1.0364
Epoch 3/29
47/47 - 1s - 17ms/step - accuracy: 0.6527 - loss: 0.9710
Epoch 4/29
47/47 - 1s - 17ms/step - accuracy: 0.6585 - loss: 0.9386
Epoch 5/29
47/47 - 1s - 16ms/step - accuracy: 0.6630 - loss: 0.9128
Epoch 6/29
47/47 - 1s - 16ms/step - accuracy: 0.6747 - loss: 0.8943
Epoch 7/29
47/47 - 1s - 17ms/step - accuracy: 0.6800 - loss: 0.8783
Epoch 8/29
47/47 - 1s - 28ms/step - accuracy: 0.6882 - loss: 0.8553
Epoch 9/29
47/47 - 1s - 29ms/step - accuracy: 0.6856 - loss: 0.8466
Epoch 10/29
47/47 - 1s - 28ms/step - accuracy: 0.7024 - loss: 0.8245
Epoch 11/29
47/47 - 1s - 26ms/step - accuracy: 0.7022 - loss: 0.8233
Epoch 12/29
47/47 - 1s - 16ms/step - accuracy: 0.7122 - loss: 0.8059
Epoch 13/29
47/47 - 1s - 16ms/step - accuracy: 0.7053 - loss: 0.8032
Epoch 14/29
47/47 - 1s - 29ms/step - accuracy: 0.7168 - loss: 0.7872
Epoch 15/29
47/47 - 1s - 17ms/step - accu

## 6. Confusion Matrix

In [74]:
# Set up 'label key' for confusion matrix (similar to activities in HAR Data)
stations = {
    0: 'BASEL', 
    1: 'BELGRADE', 
    2: 'BUDAPEST', 
    3: 'DEBILT', 
    4: 'DUSSELDORF', 
    5: 'HEATHROW', 
    6: 'KASSEL', 
    7: 'LJUBLJANA', 
    8: 'MAASTRICHT', 
    9: 'MADRID', 
    10: 'MUNCHENB', 
    11: 'OSLO', 
    12: 'SONNBLICK', 
    13: 'STOCKHOLM', 
    14: 'VALENTIA'
}

In [76]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [78]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [80]:
# Create and show confusion matrix

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  HEATHROW  MADRID  MUNCHENB  \
True                                                                        
BASEL        3491       131         3       3         8      43         1   
BELGRADE      408       624         3       5         6      45         1   
BUDAPEST       96        75         4       1         8      30         0   
DEBILT         33        33         3       5         5       3         0   
DUSSELDORF     12        10         3       1         2       1         0   
HEATHROW       27         9         0       5        13      28         0   
KASSEL          5         3         0       1         0       2         0   
LJUBLJANA      24         5         1       2         3      26         0   
MAASTRICHT      8         0         1       0         0       0         0   
MADRID        136        45         4       1         7     265         0   
MUNCHENB        7         0         0       0         0       1         0   